### Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import plotly.express as px

import copy
import seaborn as sns
import os
from scipy import stats
import datetime

%matplotlib inline

In [2]:
data_path = '/mnt/d/lighthouse/Midterm_data/'

In [3]:
# 10770229 rows × 28 columns
flights = pd.read_csv(data_path + 'flights_cleaned.csv', header = 0)
flights.drop('Unnamed: 0', axis =1, inplace = True)

In [4]:
flights

,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,...,tail_num,origin,dest,bin_mkt_unique_carrier,bin_branded_code_share,bin_mkt_carrier,bin_op_unique_carrier,year,month,date
0,2098,2098,13198,14107,1755,1753.0,-2.0,7.0,1800.0,1832.0,...,0,0,0,0,0,0,0,2019,3,25
1,2238,2238,13198,14107,2000,1955.0,-5.0,6.0,2001.0,2030.0,...,1,0,0,0,0,0,0,2019,3,25
2,2451,2451,13198,14107,540,541.0,1.0,7.0,548.0,617.0,...,2,0,0,0,0,0,0,2019,3,25
3,565,565,13198,14679,1300,1315.0,15.0,7.0,1322.0,1430.0,...,3,0,1,0,0,0,0,2019,3,25
4,1387,1387,13198,14679,1955,1952.0,-3.0,6.0,1958.0,2104.0,...,4,0,1,0,0,0,0,2019,3,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10770224,1751,1751,13198,13487,1025,1026.0,1.0,11.0,1037.0,1140.0,...,102,0,48,0,0,0,0,2019,3,25
10770225,2189,2189,13198,13495,2055,2111.0,16.0,5.0,2116.0,2240.0,...,117,0,27,0,0,0,0,2019,3,25
10770226,2470,2470,13198,14057,920,915.0,-5.0,7.0,922.0,1049.0,...,241,0,61,0,0,0,0,2019,3,25
10770227,1651,1651,13198,14107,1125,1123.0,-2.0,9.0,1132.0,1158.0,...,244,0,0,0,0,0,0,2019,3,25
